## Entity Relationship Diagram (ERD) of six tables with the necessary column names from WoundExpert

In [ ]:
import os
from sqlalchemy import inspect
from sqlalchemy.orm import relationship 
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.schema import CreateTable

folder_path = 'OneDrive/...University'
chunksize = 10
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

Base = declarative_base()


def dtype_mapping(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return Integer
    elif pd.api.types.is_string_dtype(dtype):
        return String
    elif pd.api.types.is_float_dtype(dtype):
        return Float
    elif pd.api.types.is_bool_dtype(dtype):
        return Boolean
    else:
        raise ValueError(f"Unhandled data type: {dtype}")


tables = {}
common_columns = set()

# Defining relevant columns for each file
relevant_columns = {
    'tbl_WoundAssessment.csv': ['DocumentID', 'WoundID', 'Length', 'Width', 'Depth'],
    'tbl_DocumentLibrary.csv': ['DocumentID', 'DocumentType', 'PatientID', 'Title'],
    'tbl_Wound.csv': ['DocumentID', 'WoundID', 'DateAdded'],
    'tbl_Patient.csv': ['PatientGender', 'RaceCodeList', 'patientEthnicity', 'PatientDOB'],
    'tbl_PatientNutritionAssessmentInstance.csv': ['BMI', 'DocumentID', 'VisitID', 'PatientID'],
    'tbl_SocialHistoryPatientInstances.csv': ['SmokingStatus', 'DocumentID']
    
}

# Extracting common columns
for file in files:
    if file in relevant_columns:
        common_columns.update(relevant_columns[file])

# Creating table classes for each relevant CSV file
for file in files:
    if file in relevant_columns:
        class_name = f"{file[:-4]}_table"
        chunks = pd.read_csv(os.path.join(folder_path, file), sep='|', chunksize=chunksize)
        for df in chunks:
            columns = {}

            # Assign default primary key column 'id'
            columns['id'] = Column(Integer, primary_key=True)

            # Add table name to columns
            columns['__tablename__'] = class_name

            # Add relevant columns to table
            for column_name in df.columns:
                if column_name in relevant_columns[file]:
                    col_type = dtype_mapping(df[column_name].dtype)
                    columns[column_name] = Column(col_type)

            table_class = type(class_name, (Base,), columns)
            tables[class_name] = table_class
            break


# Adding relationships for common columns
for common_col in common_columns:
    for table_name, table_class in tables.items():
        if common_col in table_class.__table__.columns:
            for related_table_name, related_table_class in tables.items():
                if common_col in related_table_class.__table__.columns and table_name != related_table_name:
                    foreign_key = ForeignKey(f"{related_table_class.__tablename__}.{common_col}")
                    relationship_name = f"{related_table_class.__tablename__}_rel"
                    setattr(table_class, relationship_name, relationship(related_table_class, foreign_keys=[foreign_key]))

# Writing the SQL file
engine = create_engine('sqlite:///dummy.db')  # Use a dummy SQLite engine to generate SQL statements
Base.metadata.create_all(engine)

output_dir = "OneDrive/WoundExpert"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, 'ERD_WoundExpert.sql')

from sqlalchemy.schema import CreateTable
with open(output_path, 'w') as f:
    for table in Base.metadata.tables.values():
        create_table_statement = CreateTable(table).compile(engine)
        f.write(str(create_table_statement))
        f.write(";\n\n")